In [1]:
!pip install -q openai
!pip install panel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [1]:
import os

OPENAI_API_KEY = str(input("Enter OpenAI API Key"))
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Enter OpenAI API Keysk-MVWIrTqwgRVzSDbgFs0jT3BlbkFJT8bEjXkP4HpYf0RC3p73


In [2]:
import openai
import panel as pn

In [3]:
def continue_conversation(messages, temperature=0):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature
    )

    return response.choices[0].message.content

In [4]:
panels = []

In [5]:
# Creating the prompt
# Read and understand it

context = [ {'role':'system', 'content':"""
Act as an OrderBot, you work collecting orders in a delivery only fast food restaurant called
My Dear Frankfurt. \
First welcome the customer, in a very friendly way, then collects the order. \
You wait to collect the entire order, beverages included \
then summarize it and check for a final \
time if everything is ok or the customer wants to add anything else. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very friendly style. \
The menu includes \
burger  12.95, 10.00, 7.00 \
frankfurt   10.95, 9.25, 6.50 \
sandwich   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
martra sausage 3.00 \
canadian bacon 3.50 \
romesco sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
vichy catalan 5.00 \
"""} ]

In [6]:
client_prompt = pn.widgets.TextInput(value="Hi", placeholder="Enter text here ... ")


In [7]:
def add_prompts_conversation(_):
    # Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    # Append to the context the user prompt
    context.append(
        {"role": "user",
         "content": f"{prompt}"}
    )

    # Get the response
    response = continue_conversation(context)

    # Add the response to the context
    context.append({"role": "assistant",
                    "content": f"{response}"})

    # Update the panels to show the conversation
    panels.append(pn.Row("User:", pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row("Assistant", pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [8]:
# Creating the panel

pn.extension()

In [9]:
button_conversation = pn.widgets.Button(name="talk")

In [10]:
interactive_conversation = pn.bind(add_prompts_conversation,
                                   button_conversation)

In [11]:
dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here ... ')
    [1] Row
        [0] Button(name='talk')
    [2] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True)